In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Recommend").getOrCreate()

In [3]:
from pyspark.ml.recommendation import ALS

In [4]:
from pyspark.ml.evaluation import RegressionEvaluator

In [5]:
df = spark.read.csv("movielens_ratings.csv",header=True,inferSchema=True)

In [6]:
df.head()

Row(movieId=2, rating=3.0, userId=0)

In [7]:
df.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [9]:
df.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [10]:
training,test = df.randomSplit([0.8,0.2])

In [11]:
als = ALS(maxIter = 5, regParam=0.01,userCol="userId",itemCol="movieId",ratingCol="rating")

In [12]:
model = als.fit(training)

In [16]:
preds = model.transform(test)

In [17]:
preds.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      1|   1.0|    26| 0.05147606|
|      1|   1.0|     3|  0.8314596|
|      1|   1.0|     5|-0.05700767|
|      1|   1.0|     4|-0.30826247|
|      1|   3.0|    25|  3.5845962|
|      3|   1.0|    17|   1.355134|
|      3|   2.0|     8|  3.0077276|
|      5|   1.0|     9|  0.9152795|
|      5|   1.0|     8|  1.3467104|
|      4|   1.0|    23| -1.1225047|
|      2|   1.0|    12|  3.7967432|
|      2|   1.0|    16|  2.0358725|
|      2|   1.0|    15|  3.4855278|
|      2|   1.0|    17|  1.4996477|
|      2|   4.0|    21|   4.327427|
|      2|   3.0|     0|  3.3691332|
|      0|   3.0|    28|-0.29878286|
|      0|   1.0|    22|  1.1962535|
|      0|   1.0|     5| 0.77513444|
|      0|   1.0|    19|  1.0820303|
+-------+------+------+-----------+
only showing top 20 rows



In [18]:
evaluator = RegressionEvaluator(metricName = "rmse",labelCol = "rating",predictionCol= "prediction")

In [21]:
rmse = evaluator.evaluate(preds)## root mean squared error

In [22]:
rmse

1.8439554900691513

In [23]:
single_user = test.filter(test["userId"]==11).select(["movieId","userId"]) 

In [24]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      9|    11|
|     19|    11|
|     40|    11|
|     43|    11|
|     66|    11|
|     71|    11|
|     78|    11|
|     81|    11|
|     82|    11|
|     86|    11|
|     94|    11|
|     99|    11|
+-------+------+



In [25]:
recommendations = model.transform(single_user)

In [26]:
recommendations.orderBy("prediction",ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     94|    11|   1.925983|
|     71|    11|  1.3858715|
|     43|    11| 0.95517164|
|     66|    11|  0.8413675|
|     86|    11|  0.6389628|
|     19|    11| 0.51505667|
|     78|    11| 0.33906066|
|     99|    11| -0.3293077|
|      9|    11|-0.86122334|
|     40|    11|-0.87442863|
|     81|    11| -1.2726616|
|     82|    11| -1.9055955|
+-------+------+-----------+

